In [20]:
import requests
import json
import pandas as pd
from datetime import datetime

In [21]:
# Replace with your actual API key
api_key = "XLC9cIQDZOgmCiMCT43D1umf6VeOFuGU"

# Base URL for the API
base_url = "https://legislation.nysenate.gov/api/3/bills/"

# mapping columns to city tracker
column_mappings = {
    "Last Updated": None,  # date tracker is exported
    "Bill Number": "basePrintNoStr",
    "Date Introed": "publishedDateTime",  # this date will be reformatted
    "Automated Status": "status_statusType",
    "Sponsor": "sponsor_member_fullName",
    "Co Sponsor Count": None,  # result.amendments.items..coSponsors.size
    "Co Sponsor Count Change": None,  # 
    "Bill Last Changed": "status_actionDate",
    "Summary": "summary",
    "Committee": "status_committeeName",
    "Co Sponsors": None,  # 
    "Notes": None,  # 
    "Laws Impacted": None,  #result.amendments.items..relatedLaws.items.AMEND.items
    "MatterTypeName": None,  # 
    "Priority Level": None  #
}

# ADD a filter for state committee

In [22]:
def fetch_bills_by_year(api_key, session_year):
    """
    Fetch bills for a given session year from the NY Senate API.
    """
    url = f"{base_url}{session_year}?key={api_key}"

    try:
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            bill_data = response.json()

            if bill_data.get("result") and bill_data["result"].get("items"):
                print(f"Fetched {len(bill_data['result']['items'])} bills for the {session_year} session.")
                return bill_data
            else:
                print(f"No bills found for session year {session_year}.")
                return None
        else:
            print(f"Failed to retrieve bills. Status code: {response.status_code}")
            print(f"Response: {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

In [23]:
def flatten_json(nested_json, parent_key='', sep='_'):
    """
    Flatten a nested JSON into a flat dictionary.
    """
    items = []
    for k, v in nested_json.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_json(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for i, item in enumerate(v):
                items.extend(flatten_json(item, f"{new_key}_{i}", sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [24]:
def get_column_data(item, column_mappings):
    """
    Create a row of data based on the column mappings.
    """
    row_data = {}

    for column, key in column_mappings.items():
        if key:
            # Check if the key exists in the flattened JSON data and add it to the row
            value = item.get(key, "")

            # If the column is 'Date Introed', format it as 'YYYY-MM-DD'
            if column == "Date Introed" and value:
                try:
                    # Convert to date format (YYYY-MM-DD)
                    formatted_date = datetime.fromisoformat(value).date()
                    row_data[column] = formatted_date
                except ValueError:
                    row_data[column] = value  # If it can't be converted, leave the original value
            else:
                row_data[column] = value
        else:
            # Handle empty columns that need to be calculated or filled manually
            row_data[column] = ""

    return row_data

In [25]:
def export_to_excel(bill_data, filename="NY_Senate_bills_data"):
    """
    Export the flattened bill data into an Excel file.
    """
    flattened_data = []

    if bill_data and 'result' in bill_data and 'items' in bill_data['result']:
        items = bill_data['result']['items']
        for item in items:
            flattened_item = flatten_json(item)
            row_data = get_column_data(flattened_item, column_mappings)
            flattened_data.append(row_data)

    # Create a DataFrame
    df = pd.DataFrame(flattened_data)

    # Export the data to Excel
    if not df.empty:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{filename}_{timestamp}.xlsx"
        df.to_excel(filename, index=False)
        print(f"Data exported to {filename}")
    else:
        print("No data to export to Excel.")

In [26]:
def main():
    session_year = 2025  # Specify the session year (e.g., 2023)
    bill_data = fetch_bills_by_year(api_key, session_year)

    if bill_data:
        # Export the flattened data to Excel
        export_to_excel(bill_data)
    else:
        print("No data retrieved from the API.")

if __name__ == "__main__":
    main()

Fetched 50 bills for the 2025 session.
Data exported to bills_data_20250319_190738.xlsx
